In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise
from keras.utils import to_categorical
import keras

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Bobby\.conda\envs\dl\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#Data corresponds to an emotion tag, a 48x48 greyscale picture flattened, and a tag indicating if the image belongs to the test set or the training set.
df = pd.read_csv('./data/fer2013.csv')

train = df[df['Usage']=='Training']
test = df[df['Usage']=='PublicTest']

#TODO: We might want to augment data with ImageDataGenerator 

#TODO: We might want to do some image pre-processing


#Splits data in training and testing, as well as formatting data to right input and output shapes

X_train = np.array(train.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48,1).astype('float32')))
X_test = np.array(test.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48,1).astype('float32')))

#Turn the array of 3D arrays into a 4D array
X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)

y_train = to_categorical(train['emotion'])
y_test =  to_categorical(test['emotion'])

#X_train = X_train / 255
#X_test = X_test / 255

#Splits off a validation set
X_val = X_train[-4000:]
y_val = y_train[-4000:]
X_train = X_train[:-4000]
y_train = y_train[:-4000]

print(y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]]


In [3]:
datagen = ImageDataGenerator(
        #rotation_range=40,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #brightness_range=0.2,
        #zca_epsilon: epsilon for ZCA whitening. Default is 1e-6.
        #zca_whitening: Boolean. Apply ZCA whitening.
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,    
        fill_mode='nearest')

In [4]:
#Define dimensions of input and output layers
D_in = X_train[0].shape
D_out = tuple(y_train[0].shape)
print(D_in)
print(D_out[0])

#Sets up model. To Bobby: Can we make this smarter so you and I can do different models?
model = Sequential()

#model.add(GaussianNoise(0.1, input_shape=(tuple(D_in))))

model.add(Conv2D(32, (3, 3), input_shape=(tuple(D_in))))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

#model.add(Dense(8, activation='softmax'))

model.add(Dense(D_out[0], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#TODO: Train networks, evaluate results.

(48, 48, 1)
7


In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 21, 21, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)        

In [6]:
callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss",patience=10)]

In [61]:
print('# Fit model on training data')

batch_size = 64

history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_val, y_val), steps_per_epoch=len(X_train) // batch_size,
    epochs=500, callbacks = callbacks)

# history = model.fit(X_train, y_train,
#                     batch_size=64,
#                     epochs=500,
#                     # We pass some validation for
#                     # monitoring validation loss and metrics
#                     # at the end of each epoch
#                     validation_data=(X_val, y_val),
#                     callbacks = callbacks
#                    )

#print('\nhistory dict:', history.history)


# Fit model on training data
Epoch 1/500
386/386 [==============================] - 27s 69ms/step - loss: 1.2421 - accuracy: 0.5302 - val_loss: 108.1487 - val_accuracy: 0.3620
Epoch 2/500
386/386 [==============================] - 27s 69ms/step - loss: 1.2280 - accuracy: 0.5339 - val_loss: 105.6871 - val_accuracy: 0.3857
Epoch 3/500
386/386 [==============================] - 26s 67ms/step - loss: 1.2181 - accuracy: 0.5399 - val_loss: 102.1234 - val_accuracy: 0.3870
Epoch 4/500
386/386 [==============================] - 26s 69ms/step - loss: 1.2017 - accuracy: 0.5442 - val_loss: 105.3555 - val_accuracy: 0.3968
Epoch 5/500
386/386 [==============================] - 27s 71ms/step - loss: 1.1970 - accuracy: 0.5516 - val_loss: 74.5438 - val_accuracy: 0.4347
Epoch 6/500
386/386 [==============================] - 27s 70ms/step - loss: 1.1806 - accuracy: 0.5540 - val_loss: 94.0540 - val_accuracy: 0.3935
Epoch 7/500
386/386 [==============================] - 27s 71ms/step - loss: 1.1757 - accur

In [62]:
import tensorflow as tf

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

AttributeError: module 'tensorflow' has no attribute 'Session'